In [34]:
import tensorflow as tf
import random
import numpy as np
import os
import time

In [35]:
betaDS = open("../datasets/b/dataset.txt", "r", encoding="utf-8").read()
vocab = sorted(set(open("../datasets/vocab.txt", "r", encoding="utf-8").read()))

print('{} songs on Beta Dataset'.format(len(betaDS)))

rdm = betaDS.split('\n\n')

random.shuffle(rdm)

text = '\n\n'.join(rdm)

1010508 songs on Beta Dataset


In [36]:
seq_length = 5
BATCH_SIZE = 128
BUFFER_SIZE = 10000
embedding_dim = 256
rnn_units = 4096
EPOCHS = 1

In [37]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [38]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [39]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1010508,), dtype=int64, numpy=array([ 1, 47, 28, ...,  2, 94, 94], dtype=int64)>

In [40]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [41]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))



M
:
9
/
8


R
:
s


In [42]:
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\nM:9/8'
b'\nR:sli'
b'pjig\nK'
b':D\n|: '
b'(f2 a)'


In [43]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [44]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\nM:9/'
Target: b'M:9/8'


In [45]:
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((128, 5), (128, 5)), types: (tf.int64, tf.int64)>

In [46]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
vocab_size

96

In [47]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [48]:
# Be sure the vocabulary size matches the `StringLookup` layers.
model = MyModel(vocab_size=len(ids_from_chars.get_vocabulary()), embedding_dim=embedding_dim, rnn_units=rnn_units)

In [49]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 5, 97) # (batch_size, sequence_length, vocab_size)


In [50]:
dataset.element_spec

(TensorSpec(shape=(128, 5), dtype=tf.int64, name=None),
 TensorSpec(shape=(128, 5), dtype=tf.int64, name=None))

In [51]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  24832     
                                                                 
 gru_2 (GRU)                 multiple                  53501952  
                                                                 
 dense_2 (Dense)             multiple                  397409    
                                                                 
Total params: 53,924,193
Trainable params: 53,924,193
Non-trainable params: 0
_________________________________________________________________


In [52]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [53]:
sampled_indices

array([65, 50, 43, 40, 14], dtype=int64)

In [54]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' | A>'

Next Char Predictions:
 b'_PIF,'


In [55]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [56]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (128, 5, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.574607, shape=(), dtype=float32)


In [57]:
tf.exp(example_batch_mean_loss).numpy()

96.98991

In [58]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [59]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [60]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [61]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [62]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

1315/1315 [==============================] - 3417s 3s/step - loss: 1.8836


In [63]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [64]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [65]:
start = time.time()
states = None
next_char = tf.constant(['M'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

M: 3/4
L:1/8
L:1/8
L:1/8
L:1/8
R:Hornpipe
K:D
ef=g
M:6/8
L:1/8
K:Em
dz2d c2:|


M:9/8
L:1/8
R:Reil
M:C
L:1/8
K:G
G3 =F
EFG  FDD2||
d2 gfe defedBB (dB) (c>B) \
| B2G2G2 :|
(g2 f2 f2 d2f e2cA GBAG|D2z2:|


M:6/8
L:1/8
R:Hornpipe
M:C
L:1/8
R:HornpipjississssississssissississsissssssisssisssissississississsisssissssisssssssissssssissssisssississsisssissssissssisssisssissssisssissssisssissississMsiisissssissssissssisssisssssissssississssissssssssssssisssssissssssssssssississssssssssisssisisississsisssississssssississssississssissssissssssissssssssssisssssssssssssssissssssisssssssssssssisssssississssssissssissssssissississississsssssssssissssssissssisssssisssssssssissssisssssssMssssisssssisssissssissssssisssssssssssissssssslissssssissississsssssssssississsssssississsssissssssssssssissssissssssnssisssssssssississsssissssssssssisssissssislisssssssssisssssssssisssissississssisssssssssssissssississssissississssssissssssssisssissssisssssssisssssissississssisssssiissississississsssisssissssssissss

In [66]:
tf.saved_model.save(one_step_model, 'one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [67]:
one_step_model = tf.saved_model.load('one_step')

In [68]:

betaDS = open("../datasets/b/dataset.txt", "r", encoding="utf-8").read()
vocab = sorted(set(open("../datasets/vocab.txt", "r", encoding="utf-8").read()))

print('{} songs on Beta Dataset'.format(len(betaDS)))

1010508 songs on Beta Dataset


In [69]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [70]:
start = time.time()
states = None
next_char = tf.constant(['M'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

M:6/8
L:1/8
K:D
c af fi>ga | b2 (ag) afd gbc'bc'>b a>edc A>cdc Bcd ABc|fBdc B2z2'2|
[ctore z2d2 (dG) DEGA | {F}G2 z2 E2|
efc gfe   d>cf d>cB AFA E2 A2 ||
(B,G,>F | D2 B,A,2B,2]


M:6/8
L:1/8
R:Hornpipe
M:C
L:1/8
K:G
(FEF) | D>D E2 G    PG/2A/2) |
  |  B A2 A G2) (c(Bcd) e3 f (g2 d2) | e2 d|cAGF|EGE DBF    D2 d2 BG A2 :|
|| (dc) |
|:f>ed2 | ce cAB c>AGd | Afed)cB | ~d2 d A3 z  | (e2d cAe-AA  bc'ba gfe  | d2de f=ed |3Aa^c edc  {ec}dB>c d2A>c B3 A2 |]


R:air
M:6/8
L: //8
Q:1/8
K:D
|:(dc)|B3 G3 |\
| d3 d2 d2 (ed) | B3 32cd g2 f | e2 A | d4  d2 B/2c/2d/2e/2) |
|[=cBA BGA  ABc dBG|g2dB BdBG|cde fgf ge fddB | c3 cecc BABA|(~B,E).D .D2:|
f|g
e|\
{d}cBA G2(BAGG G2:|
||d | c2 B c AGFA | d2 d3 d2 B2 |2GGG G2 :|
(f>gf e2 (c/d/) | e2) (ed)c Bd)(B | d3 | d2g e2 d | cdc)|d4 (dc)|BAGA A2 A2:|


M:3/8
L:1/8
L:1/8
K:A
E2 | A^Bc | A)ABA^Fe (3ABA|fg  agfe | d>ed>cde fdc BGBc|dBA=FA2G2 B A2(cB) | cAB BBBd|gafg bgfg a2 g f2  (f>ed cA {Bc}d2 cB | edBGA ABc B2d(c BAA A2:|


M:6/8
L:1/8
L:1/8
K:A
tirtiili>G g